In [1]:
# 1. Count of unique nodes in (sub)graph
# 2. Drop non-helpful categories
# 3. Performance of most similar analyses, breadth first. Stop iteration once sufficient coverage reached.


In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
'''

- Know count of (unique) articles (not :Category)
- Drop unwanted categories
- Know count of (unique) articles (not :Category)
- Iterate following:
    - Calculate main path of next level categories
    - Know count of (unique) articles in graph with selected Category nodes

OUTPUT: DataFrame with Category (wikipedia-id, db-id, title) and data: closest parent; path to mtc
'''


In [3]:
from py2neo import *
import pandas as pd

In [2]:
graph = Graph()

In [ ]:
%%time
results = graph.run('MATCH (pages:Page)-[BELONGS_TO]-(:Page {title: "Main_topic_classifications"}) RETURN pages.title')
results.data()

In [15]:
%%time
results = graph.run('MATCH (pages:Page)-[BELONGS_TO]->(:Page {title: "Main_topic_classifications"}) RETURN pages')
results.data()

Wall time: 47.8 ms


[{'pages': (_5215236:Category:Page {id: 693555, isNew: false, isRedirect: false, title: 'History'})},
 {'pages': (_5846218:Category:Page {id: 3260154, isNew: false, isRedirect: false, title: 'World'})},
 {'pages': (_5481373:Category:Page {id: 722196, isNew: false, isRedirect: false, title: 'Reference'})},
 {'pages': (_5214868:Category:Page {id: 691008, isNew: false, isRedirect: false, title: 'People'})},
 {'pages': (_5214980:Category:Page {id: 691928, isNew: false, isRedirect: false, title: 'Law'})},
 {'pages': (_5215112:Category:Page {id: 692694, isNew: false, isRedirect: false, title: 'Religion'})},
 {'pages': (_5477686:Category:Page {id: 696763, isNew: false, isRedirect: false, title: 'Education'})},
 {'pages': (_5214957:Category:Page {id: 691810, isNew: false, isRedirect: false, title: 'Philosophy'})},
 {'pages': (_5477550:Category:Page {id: 695027, isNew: false, isRedirect: false, title: 'Politics'})},
 {'pages': (_6891227:Category:Page {id: 47642171, isNew: false, isRedirect: fal

In [ ]:
%%time
results = graph.run('MATCH (pages:Category:Page)-[BELONGS_TO]->(:Page {title: "Humanities"}) RETURN pages')
results.data()

In [ ]:
%%time
results = graph.run('MATCH (pages:Category:Page)<-[BELONGS_TO]-(:Page {title: "Books"}) RETURN pages.title')
results.data()

In [6]:
%%time
# All category nodes to list with title and id, wall time: 1min 52s, len = 1 726 005
allCats = [(a["title"], a["id"]) for a in graph.nodes.match("Category")]

Wall time: 1min 55s


In [ ]:
len(allCats)

In [13]:
allCats[100:130]

[('Ethics', 691802),
 ('Abuse', 691808),
 ('Philosophy', 691810),
 ('Humorists', 691811),
 ('Emotion', 691820),
 ('Paul_McCartney', 691823),
 ('John_Lennon', 691827),
 ('Pearl_Jam', 691842),
 ('Soundgarden', 691846),
 ('Polish_monarchs', 691859),
 ('Poland', 691860),
 ('Functional_analysis', 691866),
 ('Weapons_of_mass_destruction', 691868),
 ("Thirty_Years\\'_War", 691871),
 ('Mathematical_analysis', 691875),
 ('War', 691877),
 ('Christianity', 691886),
 ('Combinatorics', 691896),
 ('Graph_theory', 691898),
 ('Operator_theory', 691908),
 ('Complex_analysis', 691914),
 ('Topology', 691921),
 ('Mercian_monarchs', 691922),
 ('Hentai', 691924),
 ('Civics', 691926),
 ('Law', 691928),
 ('Danish_monarchs', 691937),
 ('Theoretical_physics', 691938),
 ('World_music', 691940),
 ('Kalmar_Union', 691941)]

In [ ]:
%%time
# All category nodes to list with title and id, wall time: 1min 52s, len = 1 726 005
fiveCats = graph.nodes.match("Category").limit(5).to_data_frame()

In [ ]:
books = graph.run('MATCH (pages:Category:Page)<-[BELONGS_TO]-(:Page {title: "Books"}) RETURN pages.title, pages.id').to_data_frame()
books

In [24]:
%%time
allCatsDF = graph.run('MATCH (pages:Category:Page) RETURN pages.title, ID(pages)').to_data_frame()
allCatsDF

Wall time: 26.7 s


,pages.title,ID(pages)
0,Futurama,5214855
1,World_War_II,5214856
2,Programming_languages,5214857
3,Professional_wrestling,5214858
4,Algebra,5214859
...,...,...
1726000,Heydar_Ghiai_buildings_and_structures,7418639
1726001,Canadian_scientific_societies,7418640
1726002,Redirects_with_draftspace_articles,7418641
1726003,People_from_Vapi,7418642


In [25]:
First100 = allCatsDF.iloc[:100, :]
First100

,pages.title,ID(pages)
0,Futurama,5214855
1,World_War_II,5214856
2,Programming_languages,5214857
3,Professional_wrestling,5214858
4,Algebra,5214859
...,...,...
95,Sustainable_agriculture,5214950
96,Sustainability,5214951
97,Pokémon,5214952
98,War_films,5214953


In [31]:
First100["belongsToCount"] = None

<ipython-input-31-a6273a20028b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  First100["belongsToCount"] = None


In [3]:
matcher = NodeMatcher(graph)

In [16]:
%%time
# (_7319681:Category:Page {id: 52141708, isNew: false, isRedirect: false, title: 'Humanities_conferences'})

# Returns count of nodes that belong to node with ID used in get function

len(graph.match((None, matcher.get(5215236)), "BELONGS_TO"))

Wall time: 2.99 ms


46

In [33]:
%%time
# Try this to get count of BELONGS_TO for each row
# Needs ID
# count = len(graph.match((None, a), "CREATED"))

First100["belongsToCount"] = First100["ID(pages)"].apply(lambda x: len(graph.match((None, matcher.get(x)), "BELONGS_TO")))

Wall time: 188 ms


<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
graph.match((None, matcher.get(5215236)), "BELONGS_TO").first()

(_5509557)-[:BELONGS_TO {}]->(_5215236)

In [34]:
First100

,pages.title,ID(pages),belongsToCount
0,Futurama,5214855,17
1,World_War_II,5214856,114
2,Programming_languages,5214857,190
3,Professional_wrestling,5214858,34
4,Algebra,5214859,151
...,...,...,...
95,Sustainable_agriculture,5214950,175
96,Sustainability,5214951,58
97,Pokémon,5214952,43
98,War_films,5214953,45


In [35]:
First100k = allCatsDF.iloc[:100000, :]

In [36]:
%%time
First100k["belongsToCount"] = First100k["ID(pages)"].apply(lambda x: len(graph.match((None, matcher.get(x)), "BELONGS_TO")))

Wall time: 11min 34s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
First100k

,pages.title,ID(pages),belongsToCount
0,Futurama,5214855,17
1,World_War_II,5214856,114
2,Programming_languages,5214857,190
3,Professional_wrestling,5214858,34
4,Algebra,5214859,151
...,...,...,...
99995,Unassessed_Professional_sound_production_articles,5687177,0
99996,National_Basketball_Association_templates,5687178,10
99997,Start-Class_Professional_sound_production_arti...,5687179,0
99998,Digestive_system_stubs,5687180,58


In [38]:
First1000 = allCatsDF.iloc[:1000, :]

In [40]:
First10k = allCatsDF.iloc[:10000, :]

In [41]:
%%time
First1000["belongsToCount"] = First1000["ID(pages)"].apply(lambda x: len(graph.match((None, matcher.get(x)), "BELONGS_TO")))

Wall time: 7.6 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
%%time
First10k["belongsToCount"] = First10k["ID(pages)"].apply(lambda x: len(graph.match((None, matcher.get(x)), "BELONGS_TO")))

Wall time: 1min 9s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# First10k["belongsToCount"] = First10k["ID(pages)"].apply(lambda x: len(graph.match((None, matcher.get(x)), "BELONGS_TO")))
# Time is linear with count of x, ~0,007s per row

In [43]:
len(allCatsDF)

1726005

In [44]:
allCatsDF.iloc[:100, :]

,pages.title,ID(pages)
0,Futurama,5214855
1,World_War_II,5214856
2,Programming_languages,5214857
3,Professional_wrestling,5214858
4,Algebra,5214859
...,...,...
95,Sustainable_agriculture,5214950
96,Sustainability,5214951
97,Pokémon,5214952
98,War_films,5214953


In [50]:
First100k.sort_values(by=["belongsToCount"], ascending = False).head(100)

,pages.title,ID(pages),belongsToCount
40123,All_articles_lacking_sources,5564515,196544
85268,All_articles_to_be_expanded,5652248,128179
47468,All_orphaned_articles,5578958,123468
20054,Surnames,5514656,50878
33197,American_films,5548547,46921
13169,All_pages_needing_cleanup,5502857,35981
178,Albums_by_artist,5215033,21434
25547,Indian_films,5528339,20507
7201,English_footballers,5484877,19552
30371,1988_births,5540261,15834


In [51]:
len(First100k.loc[First100k["belongsToCount"] == 0])

8530

In [53]:
First100k.loc[First100k["belongsToCount"] == 0].head(40)

,pages.title,ID(pages),belongsToCount
23,Middle-earth_places,5214878,0
200,Smashing_Pumpkins_albums,5215055,0
238,Vocalists,5215093,0
255,Homosexuality,5215110,0
265,Chubu_region,5215120,0
266,Kanto_region,5215121,0
268,Tohoku_region,5215123,0
269,Chugoku_region,5215124,0
276,Popular_music_artists,5215131,0
369,Southwest_Asian_countries,5215224,0


In [59]:
allCatsDF.loc[allCatsDF["pages.title"] == "Finnish_film_directors"]

,pages.title,ID(pages)
113679,Finnish_film_directors,5715057


In [60]:
len(graph.match((None, matcher.get(5715057)), "BELONGS_TO"))

116

In [62]:
%%time
First100["Parents"] = First100["ID(pages)"].apply(lambda x: len(graph.match((matcher.get(x), None), "BELONGS_TO")))

Wall time: 2.41 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [63]:
First100

,pages.title,ID(pages),belongsToCount,Parents
0,Futurama,5214855,17,22
1,World_War_II,5214856,114,14
2,Programming_languages,5214857,190,4
3,Professional_wrestling,5214858,34,3
4,Algebra,5214859,151,1
5,Anime,5214860,28,10
6,Abstract_algebra,5214861,156,2
7,Mathematics,5214862,31,5
8,Linear_algebra,5214863,299,1
9,Calculus,5214864,59,2


In [ ]:
# List of all categories
# Drop categories with name containg "by_year”, “of_the_year”, “List_of”, or “_in_” OR starting with number OR containing "All_", "Stub_"
# Drop categories that belong to "Wikipedia_soft_redirected_categories" / Or categories with zero children
# Add info of how many child-pages (~4h)

# Add list of categories to handle:
# 1. MTC-1 cleaned children
# 2. MTC-2 children 